In [1]:
from function import *
from data_preparation import *
from evaluation import *

from openpyxl import Workbook
import pandas as pd
import numpy as np

# Transform data

In [2]:
discrete_file = pd.ExcelFile('../dataset/screening_smsf_actives.xlsx')
updated_binary_label = discrete_file.parse('Keck_Pria_Retest')
# transform_data('../dataset/lc123_keckdata.sdf', '../dataset/keck_updated_complete.csv', updated_binary_label)

# Split Unique Data and Complement Data

In [3]:
mol_whole = updated_binary_label['Molecule'].tolist()
print len(mol_whole)

temp = pd.read_csv('../dataset/keck_updated_complete.csv')
smaller = set(temp['molecule ID(RegID)'].tolist())
print len(smaller)

output_file = open('../dataset/complement_data.csv', 'w')
i = 0
for m in mol_whole:
    if m not in smaller:
        i += 1
        print >> output_file, m
print i

74794
62608
10077


# Split data into k-fold

In [4]:
k = 5
output_file_list = ['../dataset/{}_fold_split_data/fold_1_data.csv.gz'.format(k),
                    '../dataset/{}_fold_split_data/fold_2_data.csv.gz'.format(k),
                    '../dataset/{}_fold_split_data/fold_3_data.csv.gz'.format(k),
                    '../dataset/{}_fold_split_data/fold_4_data.csv.gz'.format(k),
                    '../dataset/{}_fold_split_data/fold_5_data.csv.gz'.format(k)]
split_data('../dataset/keck_updated_complete.csv', output_file_list, k)

In [5]:
k = 10
output_file_list = ['../dataset/{}_fold_split_data/fold_1_data.csv.gz'.format(k),
                    '../dataset/{}_fold_split_data/fold_2_data.csv.gz'.format(k),
                    '../dataset/{}_fold_split_data/fold_3_data.csv.gz'.format(k),
                    '../dataset/{}_fold_split_data/fold_4_data.csv.gz'.format(k),
                    '../dataset/{}_fold_split_data/fold_5_data.csv.gz'.format(k),
                    '../dataset/{}_fold_split_data/fold_6_data.csv.gz'.format(k),
                    '../dataset/{}_fold_split_data/fold_7_data.csv.gz'.format(k),
                    '../dataset/{}_fold_split_data/fold_8_data.csv.gz'.format(k),
                    '../dataset/{}_fold_split_data/fold_9_data.csv.gz'.format(k),
                    '../dataset/{}_fold_split_data/fold_10_data.csv.gz'.format(k)]
split_data('../dataset/keck_updated_complete.csv', output_file_list, k)

# Merge data from splitting folds to form training and testing

In [6]:
train_pd = read_merged_data(output_file_list[:9])
test_pd = read_merged_data([output_file_list[9]])

# Test feature- and label- extraction

In [7]:
print train_pd.dtypes

X_train, y_train = extract_feature_and_label(train_pd)
X_test, y_test = extract_feature_and_label(test_pd)

Unnamed: 0            object
molecule ID(RegID)    object
library               object
existing SMILES       object
generated SMILES      object
1024_fingerprint      object
Pria_SSB_%INH         object
true_label            object
dtype: object


# Test evaluation metrics

In [8]:
y_actual = np.eye(4)
y_pred = np.eye(4)
y_pred[2] = np.array([0, 0, 1, 1])
y_pred[3] = np.array([0, 0, 1, 0])

for i in range(4):
    print roc_auc_multi(y_actual, y_pred, [i], np.mean)
    print roc_auc_multi(y_actual, y_pred, [i], np.median)
    print enrichment_factor_multi(y_actual, y_pred, 0.5)
    print

1.0
1.0
[[1.0, 2.0], [1.0, 2.0], [1.0, 2.0], [1.0, 2.0]]

1.0
1.0
[[1.0, 2.0], [1.0, 2.0], [1.0, 2.0], [1.0, 2.0]]

0.833333333333
0.833333333333
[[1.0, 2.0], [1.0, 2.0], [1.0, 2.0], [1.0, 2.0]]

0.333333333333
0.333333333333
[[1.0, 2.0], [1.0, 2.0], [1.0, 2.0], [1.0, 2.0]]



# Get updated data, and give some analysis

In [9]:
discrete_file = pd.ExcelFile('../dataset/screening_smsf_actives.xlsx')
continuous_file = pd.ExcelFile('../dataset/screening_smsf_continuous.xlsx')

print discrete_file.sheet_names
print continuous_file.sheet_names

[u'Keck_Pria_Retest', u'Keck_Pria_FP', u'Keck_RMI', u'Xing_MTDH_Retest', u'Xing_MTDH_DR']
[u'Keck_Pria_Primary', u'Keck_RMI_cdd', u'Xing_MTDH_cdd']


In [10]:
binary_data = discrete_file.parse('Keck_Pria_Retest')
print 'reset binary labels:'
print binary_data.dtypes

print

continuous_data = continuous_file.parse('Keck_Pria_Primary')
print 'cts labels:'
print continuous_data.dtypes

reset binary labels:
Molecule               object
Keck_Pria_AS_Retest     int64
dtype: object

cts labels:
Molecule                 object
Keck_Pria_Continuous    float64
dtype: object


# List all active compounds (updated version)

In [11]:
print binary_data[binary_data['Keck_Pria_AS_Retest']>0]

            Molecule  Keck_Pria_AS_Retest
576    SMSSF-0015261                    1
7315   SMSSF-0021761                    1
13763  SMSSF-0027944                    1
16138  SMSSF-0030209                    1
16803  SMSSF-0030826                    1
16837  SMSSF-0030860                    1
16901  SMSSF-0030922                    1
16907  SMSSF-0030928                    1
19073  SMSSF-0032984                    1
25686  SMSSF-0039184                    1
26232  SMSSF-0039974                    1
26511  SMSSF-0040244                    1
29310  SMSSF-0042907                    1
29965  SMSSF-0043541                    1
30010  SMSSF-0043585                    1
30569  SMSSF-0044122                    1
30603  SMSSF-0044155                    1
30614  SMSSF-0044166                    1
30661  SMSSF-0044210                    1
30755  SMSSF-0044297                    1
30756  SMSSF-0044297                    1
30888  SMSSF-0044424                    1
31008  SMSSF-0044540              